AUCell Scoring

## AUCell Function

In [ ]:
import os, re, gc
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse as sp
from multiprocessing import cpu_count
from pyscenic.aucell import aucell
from ctxcore.genesig import GeneSignature

# Inputs
MIN_GENES  = 1                
SEED       = 777
MAX_MB_PER_CHUNK = 800        

# Helper functions
def _prep_df(df: pd.DataFrame) -> pd.DataFrame:
    need = {"gene", "CellType_Core", "TF"}
    missing = need - set(df.columns)
    if missing:
        raise ValueError(f"Regulon df missing columns: {missing}")
    df = df.copy()
    df["gene"] = df["gene"].astype(str)
    df["CellType_Core"] = df["CellType_Core"].astype(str).str.replace(r"\s+", "_", regex=True)
    keep_cols = ["gene", "CellType_Core", "TF"] + (["Organ"] if "Organ" in df.columns else [])
    df = df[keep_cols].dropna(subset=["gene", "CellType_Core", "TF"])
    return df

def _group_genes(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df.groupby(["Organ", "CellType_Core"])["gene"]
          .apply(lambda s: sorted(pd.unique(s)))
          .reset_index()
    )

def _normalize_inplace(adata_sub, sample_key="Sample", threshold=15000, per_sample=4000, random_state=0):

    if sample_key not in adata_sub.obs:
        raise KeyError(f"obs['{sample_key}'] not found")

    if adata_sub.n_obs > threshold:
        rng = np.random.default_rng(random_state)
        keep_ids = []
        for sample, df in adata_sub.obs.groupby(sample_key):
            ids = df.index.to_numpy()
            take = min(len(ids), per_sample)
            choose = rng.choice(ids, size=take, replace=False) if take < len(ids) else ids
            keep_ids.append(choose)
        keep_ids = np.concatenate(keep_ids)
        keep_mask = adata_sub.obs_names.isin(keep_ids)
        adata_sub = adata_sub[keep_mask].copy()

    if sp.issparse(adata_sub.X):
        adata_sub.X = adata_sub.X.tocsr().astype(np.float32)
    else:
        adata_sub.X = adata_sub.X.astype(np.float32)

    sc.pp.normalize_total(adata_sub, target_sum=1e4, inplace=True)
    sc.pp.log1p(adata_sub)

    return adata_sub

def _choose_chunk_size(n_cells: int, n_genes: int, max_mb: int = MAX_MB_PER_CHUNK) -> int:
    bytes_per_cell = max(1, n_genes) * 4
    max_bytes = max_mb * 1024 * 1024
    return max(1, min(n_cells, int(max_bytes // bytes_per_cell)))

def _aucell_scores(adata_sub, geneset, seed=SEED, auc_max_rank=2000) -> pd.Series:
    sig = GeneSignature("sig", list(geneset))
    n_cells, n_genes = adata_sub.n_obs, adata_sub.n_vars
    chunk_size = _choose_chunk_size(n_cells, n_genes)
    parts = []
    for i0 in range(0, n_cells, chunk_size):
        i1 = min(i0 + chunk_size, n_cells)
        X = adata_sub.X[i0:i1]
        if sp.issparse(X):
            arr = X.toarray().astype(np.float32, copy=False)
        else:
            arr = np.asarray(X, dtype=np.float32, order="C")
        ex = pd.DataFrame(arr, columns=adata_sub.var_names, index=adata_sub.obs_names[i0:i1])
        # Pass the fixed aucMaxRank=2000
        sc_chunk = aucell(ex, [sig], num_workers=1, seed=seed, auc_threshold= (auc_max_rank/len(adata_sub.var))).iloc[:, 0]
        parts.append(sc_chunk)
        del X, arr, ex, sc_chunk
        gc.collect()
    return pd.concat(parts)

def _sanitize_filename(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9._-]+", "_", s)



# Master function

def run_tf(tf_df: pd.DataFrame, adata, OUT_ROOT, aucMax=1000):
    tf_df = _prep_df(tf_df)
    combos = _group_genes(tf_df)
    tf_name = tf_df["TF"].astype(str).unique()[0]


    organ_obs = adata.obs["Organ"].astype(str)
    cell_obs  = adata.obs["CellType_Core"].astype(str).str.replace(r"\s+", "_", regex=True)

    out_dir = os.path.join(OUT_ROOT, "regulon_AUCELL", f"{aucMax}", f"{tf_name}")
    os.makedirs(out_dir, exist_ok=True)

    i = 0
    for _, row in combos.iterrows():
        i += 1
        organ = row["Organ"]
        cell  = row["CellType_Core"]
        gene_list = list(row["gene"])
        genes = [g for g in gene_list if g in adata.var_names]

        print(
            f"[{i}/{len(combos)}] TF = {tf_name} | Organ={organ} | CellType_Core={cell} | "
            f"present={len(genes)}/{len(gene_list)}",
            flush=True
        )
        if len(genes) < MIN_GENES:
            continue

        mask = (organ_obs == organ) & (cell_obs == cell)
        if not np.any(mask):
            continue

        n_cells_subset = int(mask.sum())
        print(f"    {tf_name} | {organ} | {cell} --- {n_cells_subset} cells - normalizing", flush=True)
        ad_sub = adata[mask].copy()
        ad_sub = _normalize_inplace(ad_sub)

        print(f"    {tf_name} | {organ} | {cell} --- {ad_sub.n_obs} - scoring", flush=True)
        scores = _aucell_scores(ad_sub, genes, auc_max_rank=aucMax)
        scores = scores.reindex(ad_sub.obs_names)

        out_df = pd.DataFrame({
            "spot": ad_sub.obs_names.astype(str).values,
            "score": scores.values,
            "Organ": organ,
            "CellType_Core": cell,
            "auc_max_rank": aucMax,
            "TF": tf_name
        })
        fname = f"{_sanitize_filename(organ)}__{_sanitize_filename(cell)}__{aucMax}.csv"
        out_path = os.path.join(out_dir, fname)
        out_df.to_csv(out_path, index=False)
        print(f"    {tf_name} | {organ} | {cell} - saved", flush=True)

        del ad_sub, scores, out_df
        gc.collect()



# Run

In [ ]:
import scanpy as sc
import pandas as pd
import os

OUT_ROOT   = "/project/nchevrier/projects/clevenger1/Projects/ArraySeq/General_Analysis/Revision_Analysis/scenic/Celltypes/AUC_bothIrf1Stat1"
Regulon_dir = "/project/nchevrier/projects/clevenger1/Projects/ArraySeq/General_Analysis/Revision_Analysis/scenic/Celltypes/Analysis/IRF1_Stat1_Regulons"
aucN = 1000

adata = sc.read_h5ad("/project/nchevrier/projects/clevenger1/Projects/ArraySeq/General_Analysis/CellTypeDEGs/Objects/adata_filtered_Celltypes_OrganCoordinates.h5ad")

Irf1_df   = pd.read_csv(os.path.join(Regulon_dir, "Irf1_Regulons_OrganxCore.csv"))
Stat1_df  = pd.read_csv(os.path.join(Regulon_dir, "Stat1_Regulons_OrganxCore.csv"))
Shared_df = pd.read_csv(os.path.join(Regulon_dir, "Shared_Regulons_OrganxCore.csv"))

run_tf(Stat1_df, adata, aucMax=aucN, OUT_ROOT = OUT_ROOT)
run_tf(Irf1_df,  adata, aucMax=aucN, OUT_ROOT = OUT_ROOT)
run_tf(Shared_df,  adata, aucMax=aucN, OUT_ROOT = OUT_ROOT)